<div style='background-color: #87ceeb;
    border: 0.5em solid black;
    border-radius: 0.5em;
    padding: 1em;'>
    <h2>Activité</h2>
    <h1>Carte de vigilance météorologique</h1>
</div>

Les cartes de vigilance de Météo France permettent, depuis 2001, d'alerter la population avant la survenue de divers phénomènes tels que les vents violents, les fortes précipitations, les orages, les avalanches, etc. Elles ont été imaginées à la suite des deux tempêtes qui ont traversé la France les 26 et 27 décembre 1999 et ont tué 92 personnes.

Si elles avaient existé à l'époque, voici quelles auraient été les cartes de vigilance des 26 et 27 décembre 1999 :

<img src='https://ntoulzac.github.io/Cours-NSI-Terminale/api/images/vigilance_1999-12-26.svg' width="49%" style="display:inline-block;">
<img src='https://ntoulzac.github.io/Cours-NSI-Terminale/api/images/vigilance_1999-12-27.svg' width="49%" style="display:inline-block;">

*OpenDataSoft* met à disposition une API permettant de connaître, pour chaque département de l'Hexagone, la couleur et le ou les risques associés.

Pour obtenir au format JSON l'ensemble des informations disponibles, il suffit d'interroger l'API en se rendant à l'adresse [https://public.opendatasoft.com/api/records/1.0/search/?dataset=vigilance-meteorologique&rows=100](https://public.opendatasoft.com/api/records/1.0/search/?dataset=vigilance-meteorologique&rows=100).

**(1)** ✏️ Après avoir cliqué sur le lien permettant d'interroger l'API et avoir affiché les données renvoyées par l'API sous forme de dictionnaire, déterminer :
- le nombre de clés que possède le dictionnaire,
- quel est le type de la valeur associée à la clé `records`, combien d'éléments elle contient et de quel type sont ces éléments.

On a isolé un extrait des données concernant un département :

```json
{"datasetid": "vigilance-meteorologique",
 "recordid": "57e3f6dfb550022b892dba19b4215ce6fad943d2",
 "fields": {"nom_dept": "NORD",
            "dep": "59",
            "geo_point_2d": [50.4475936458, 3.21550472852],
            "couleur": "Jaune",
            "geo_shape": {"type": "MultiPolygon", "coordinates": [###_trop_long_pour_être_recopié_###]]},
            "risque_valeur": "Orages"},
 "geometry": {"type": "Point",
              "coordinates": [3.21550472852, 50.4475936458]},
 "record_timestamp": "1970-01-01T00:00:00+00:00"}
```

**(2)** ✏️ Si on suppose que ces données sont stockées dans un dictionnaire appelé `donnees`, déterminer quelles expressions
permettent d'obtenir :
- le numéro du département,
- la couleur correspondant au risque identifié dans le département.

**(3)** 💻 Écrire la spécification de la fonction `interroger_API_vigilance_meteo`.

In [ ]:
import requests

In [ ]:
COULEURS = {'Vert': '#008000', 'Jaune': '#ffff00', 'Orange': '#ff5e13', 'Rouge': '#ff0000'}

In [ ]:
def interroger_API_vigilance_meteo():
    """
    Détermine, pour chaque département, sa couleur sur la carte de vigilance de Météo France.
    - Sortie : dico (dictionnaire dont les clés sont les numéros des départements et la valeur associée
                     est le code RGB correspondant à la couleur du département)
    """
    url = "https://public.opendatasoft.com/api/records/1.0/search/?dataset=vigilance-meteorologique&rows=100"
    reponse = requests.get(url)
    reponse = reponse.json()
    dico = {elem['fields']['dep'] : COULEURS[elem['fields']['couleur']] for elem in reponse['records']}
    return dico

La cellule suivante met à notre disposition une procédure `creer_svg_departements` qui permet de créer une carte des départements de l'Hexagone en couleurs et au format SVG. La procédure prend deux ou trois paramètres d'entrée :
- un dictionnaire qui associe à chaque numéro de département un code couleur RGB,
- une chaîne de caractères qui est le nom de fichier SVG dans lequel stocker la carte,
- un booléen (optionnel,  valant `False` par défaut) qui permet de décider d'afficher ou pas la carte dans le carnet *Jupyter*.

In [ ]:
from carte_france import creer_svg_departements

**(4)** ✏️ 💻 Écrire des lignes de code permettant d'enregistrer dans un fichier `vigilance.svg` la carte de vigilance de Météo France et de l'afficher dans le carnet *Jupyter*.

In [ ]:
dictionnaire_couleurs = interroger_API_vigilance_meteo()
creer_svg_departements(dictionnaire_couleurs, 'vigilance.svg', True)

<div style='background-color: #87ceeb;
    border-radius: 0.5em;
    padding: 1em;'>
    <h2>Pour aller plus loin...</h2>
</div>

**(5)** 💻 Modifier la définition de la fonction `interroger_API_vigilance_meteo` pour qu'elle renvoie un dictionnaire :
- dont les clés sont les numéros des départements,
- et dont les valeurs associées aux clés sont des dictionnaires dont les clés sont `'nom_dep'` (nom du département), `'couleur'` (couleur du département), `'contour'` (limites géographiques du département au format GeoJSON) et, dans le cas où le département n'est pas vert, `'risque'` (type de risque identifié dans le département).

In [ ]:
def interroger_API_vigilance_meteo():
    """
    Détermine, pour chaque département, son nom, sa couleur et son risque sur la carte de vigilance de Météo France.
    - Sortie : dico (dictionnaire dont les clés sont les numéros des départements et la valeur associée
                     est un dictionnaire dont les clés sont 'nom_dep', 'couleur', 'contour' (limites départementales
                     au format GeoJSON) et eventuellement 'risque')
    """
    url = "https://public.opendatasoft.com/api/records/1.0/search/?dataset=vigilance-meteorologique&rows=100"
    reponse = requests.get(url)
    reponse = reponse.json()
    dico = {}
    for elem in reponse['records']:
        donnees = elem['fields']
        dico[donnees['dep']] = {'nom_dep': donnees['nom_dept'],
                                'couleur': donnees['couleur'],
                                'contour': donnees['geo_shape']}
        if donnees['couleur'] != 'Vert':
            dico[donnees['dep']]['risque'] = f"risque {donnees['risque_valeur'].lower()}"
    return dico

⚠️ La suite de l'activité utilise le module `folium`, qui peut être installé en exécutant la cellule ci-dessous.

In [ ]:
import sys
!{sys.executable} -m pip install folium

**(6)** 💻 Tester la fonction `interroger_API_vigilance_meteo` en exécutant la cellule suivante :

In [ ]:
import folium

def fonction_couleur(coul):
    def fonction_de_style(x):
        return {'color': '#000000', 'weight': 1, 'fillColor': coul, 'fillOpacity': 1}
    return fonction_de_style

COULEURS = {'Vert': '#008000', 'Jaune': '#ffff00', 'Orange': '#ff5e13', 'Rouge': '#ff0000'}

def creer_carte_vigilance(dico):
    """
    Crée une carte de vigilance Météo France via le module folium.
    - Entrée : dico (dictionnaire dont les clés sont les numéros des départements et la valeur associée
                     est un dictionnaire dont les clés sont 'nom_dep', 'couleur', 'contour' (limites départementales
                     au format GeoJSON) et eventuellement 'risque')
    - Sortie : carte (instance de la classe folium.Map)
    """
    carte = folium.Map(location=(46.513 ,2.576), tiles=None, zoom_start=6)
    for cle in dico:
        contour = dico[cle]['contour']
        if dico[cle]['couleur'] != 'Vert':
            message = f"{dico[cle]['nom_dep']} {dico[cle]['risque']}"
        else:
            message = None
        fonc_coul = fonction_couleur(COULEURS[dico[cle]['couleur']])
        folium.GeoJson(contour, tooltip=message, style_function=fonc_coul).add_to(carte)
    return carte

creer_carte_vigilance(interroger_API_vigilance_meteo())

**(6)** 💻 Modifier la définition de la fonction `creer_carte_vigilance` de sorte que, lors du survol d'un département jaune, orange ou rouge avec la souris, le nom du département apparaisse en gras et le risque apparaisse en italique sur la ligne en dessous.

In [ ]:
import folium

def fonction_couleur(coul):
    def fonction_de_style(x):
        return {'color': '#000000', 'weight': 1, 'fillColor': coul, 'fillOpacity': 1}
    return fonction_de_style

COULEURS = {'Vert': '#008000', 'Jaune': '#ffff00', 'Orange': '#ff5e13', 'Rouge': '#ff0000'}

def creer_carte_vigilance(dico):
    carte = folium.Map(location=(46.513 ,2.576), tiles=None, zoom_start=6)
    for cle in dico:
        contour = dico[cle]['contour']
        if dico[cle]['couleur'] != 'Vert':
            message = f"<b>{dico[cle]['nom_dep']}</b><br><i>{dico[cle]['risque']}</i>"
        else:
            message = None
        fonc_coul = fonction_couleur(COULEURS[dico[cle]['couleur']])
        folium.GeoJson(contour, tooltip=message, style_function=fonc_coul).add_to(carte)
    return carte

creer_carte_vigilance(interroger_API_vigilance_meteo())